## Ampersand placement and recovery
### Adding ampersands during encryption
We can calculate the number of ampersands we need to add using only the `key` and the `message`.

In [23]:
message = 'Luke, I am your father'
key = 3

In [24]:
number_of_ampersands = key - len(message)%key
number_of_ampersands

2

In [25]:
padded_message = message + '&'*number_of_ampersands
print(padded_message)

Luke, I am your father&&


Now we want to create an array containing the message so we can encrypt the message.

In [26]:
import numpy as np

In [27]:
list_of_characters = [char for char in padded_message]
array_of_characters = np.array(list_of_characters)
array_of_characters

array(['L', 'u', 'k', 'e', ',', ' ', 'I', ' ', 'a', 'm', ' ', 'y', 'o',
       'u', 'r', ' ', 'f', 'a', 't', 'h', 'e', 'r', '&', '&'], dtype='<U1')

In [28]:
reshaped_array = np.reshape(array_of_characters,(len(padded_message)//key,key))
reshaped_array

array([['L', 'u', 'k'],
       ['e', ',', ' '],
       ['I', ' ', 'a'],
       ['m', ' ', 'y'],
       ['o', 'u', 'r'],
       [' ', 'f', 'a'],
       ['t', 'h', 'e'],
       ['r', '&', '&']], dtype='<U1')

In [29]:
encrypted_array = np.reshape(np.transpose(reshaped_array),(1,len(padded_message)))
encrypted_array

array([['L', 'e', 'I', 'm', 'o', ' ', 't', 'r', 'u', ',', ' ', ' ', 'u',
        'f', 'h', '&', 'k', ' ', 'a', 'y', 'r', 'a', 'e', '&']],
      dtype='<U1')

We can read off the encrypted message by stepping though the 0th row of the array and ignoring any ampersands we come across.

In [30]:
encrypted_message = ''
for char in encrypted_array[0]:
    if char != '&':
        encrypted_message += char
print(encrypted_message)

LeImo tru,  ufhk ayrae


### Where to put ampersands when decrypting
When decrypting a message using a given key, we need to know where to put the ampersands (end of certain rows).  This means we need to know which rows to add ampersands to and which not to.

In [31]:
num_rows_with_ampersands = key - len(encrypted_message)%key
num_rows_with_ampersands

2

In [32]:
num_rows_no_ampersands = len(encrypted_message)%key
num_rows_no_ampersands

1

The following code will create a list of Boolean values that indicate if the row with the current index needs an ampersand or not.

In [33]:
needs_ampersand = [False for i in range(num_rows_no_ampersands)]
for i in range(num_rows_with_ampersands):
    needs_ampersand.append(True)
needs_ampersand

[False, True, True]

For each row in the array, we will check whether it needs an ampersand by consulting the above list.  Instead of starting with a numpy array, we will start with a list of lists of blank string, but we can still think about it as having rows and columns.

In [34]:
num_of_columns = len(encrypted_message)//key
if len(encrypted_message)%key != 0:
    num_of_columns += 1
encrypted_message_list = [['' for i in range(num_of_columns)] for i in range(key)]
encrypted_message_list

[['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '']]

In [37]:
for i in range(key):
    if needs_ampersand[i] == True:
        print('fill in row with an ampersand at the end')
    else:
        print('fill in row without ampersand')

fill in row without ampersand
fill in row with an ampersand at the end
fill in row with an ampersand at the end
